In [ ]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict

%matplotlib inline

In [ ]:
env = gym.make('MountainCar-v0')
env.reset()
print(env.action_space)
print(env.observation_space.low, env.observation_space.high)
print(env.action_space.sample())
print(env.step(0))

In [ ]:
env.reset()

action = {
    'left': 0,
    'stop': 1,
    'right': 2,
}
R = []
t = 0
rm = -1
while True:
    env.render()
    
    if t < 30:
        state, reward, done, info = env.step(action['left'])
    elif t < 50:
        state, reward, done, info = env.step(action['right'])
    elif t < 80:
        state, reward, done, info = env.step(action['left'])
    else:
        state, reward, done, info = env.step(action['right'])
    if done:
        print('Done')
        break
    rm = max(rm, state[0])
    t += 1
    time.sleep(0.05)
print(rm)
print(R)
time.sleep(5)
env.close()

In [ ]:
def get_state(state):
    return (round(state[0], 2), round(state[1], 3))

In [ ]:
def init_qtable():
    return {}

In [ ]:
st = env.reset()
print(st)
print(get_state(st))

In [ ]:
Episode = 4096 * 1000
Alpha = 0.5
Gamma = 0.9
Epsilon = 0.2

In [ ]:
def select(qtable, state):
    row = qtable.get(state, [0, 0, 0])
    mx = max(row)
    res = []
    for index in range(len(row)):
        if row[index] == mx:
            res.append(index)
    return random.choice(res)

In [ ]:
def select_eps(qtable, state, epsilon):
    if random.random() <= epsilon or state not in qtable:
        return env.action_space.sample()
    else:
        return select(qtable, state)

In [ ]:
# q_learning
qtable = init_qtable()
for _ in range(Episode):
    st = get_state(env.reset())
    done = False
    while not done:
        act = select_eps(qtable, st, Epsilon)
        state, reward, done, info = env.step(act)
        new_st = get_state(state)
        new_act = select(qtable, new_st)
        if st not in qtable:
            qtable[st] = [0, 0, 0]
        if new_st not in qtable:
            qtable[new_st] = [0, 0, 0]
        qtable[st][act] += Alpha * (reward + Gamma * qtable[new_st][new_act] - qtable[st][act])
        st = new_st
        act = new_act
        
        if _ % 1024 == 0:
            env.render()
            time.sleep(0.05)
        
        if done and len(info) == 0:
            print('OK')

In [ ]:
print(_)

In [ ]:
env.close()

In [ ]:
# Sarsa
qtable = init_qtable()
for _ in range(Episode):
    st = get_state(env.reset())
    act = select(qtable, st)
    done = False
    #print('Episode {}'.format(_))

    while not done:
        state, reward, done, info = env.step(act)
        
        nst = get_state(state)
        nact = select(qtable, nst)
        qtable[st][act] += Alpha * (reward + Gamma * qtable[nst][nact] - qtable[st][act]) 
        
        st = nst
        act = nact
            
        if done and len(info) == 0:
            print('Ok')
            
        if _ % 4096 == 0:
            pass
#             env.render()
#             time.sleep(0.05)
        
env.close()